# TF 변환
전처리는 계산 비용이 크기 때문에 학습과 동시에 진행하는 것 보다는 사전에 처리하는 것이 속도를 크게 높일 수 있음.  
즉 데이터가 학습하는 동안 에포크마다 진행되는 것이 아니라 **학습하기 전에 샘플마다 한 번씩** 전처리하는 것이 좋음.  
- 데이터가 RAM에 다 들어갈 수 있을 만큼 작다면 **cache()** 메서드를 사용할 수 있음
- 데이터가 아주 크면 **아파치 빔, 스파크** 같은 도구가 도움이 됨.

이런 도구로 대규모 데이터셋에서 효율적인 데이터 처리 파이프라인을 수행할 수 있으므로 학습 전에 학습 데이터를 전처리할 수 있음.  
하지만 도구를 쓰면 나중에 모델을 배포할 때 귀찮아진다고 함.  
> 전처리 연산을 딱 한번만 정의하려면 **TF 변환** 을 사용하면 된다고 함.  
TF 변환은 텐서플로 모델 상품화를 위한 엔드-투-엔드 플랫폼 TFX(tensorflow extended)의 일부분임.

In [24]:
import tensorflow as tf
import tensorflow_transform as tft

In [48]:
tf.compat.v1.disable_eager_execution()
def preprocess(inputs):
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]
    
    median_age_tensor = tf.constant(median_age, dtype=tf.int64)
    ocean_poximity_tensor = tf.constant(ocean_proximity)
    
    standardized_age = tft.scale_to_z_score(median_age_tensor)
    ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_poximity_tensor)
    return {
        "standardized_median_age": standardized_age,
        "ocean_proximity_id": ocean_proximity_id
    }

In [9]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [10]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [19]:
housing = load_housing_data()
housing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [57]:
preprocessed_housing = preprocess(housing)
preprocessed_housing["standardized_median_age"]

<tf.Tensor 'scale_to_z_score_11/SelectV2:0' shape=(20640,) dtype=float32>

?????????????????????????  
이걸 어떻게 사용하라는 거지??